# Advent of Code: Day 13

In [125]:
using Base.Iterators
using DelimitedFiles
using Match
using Underscores

function parse_data(fn)
  lines = readlines(fn)
  seperatorIndex = findall(v -> v == "", lines)[1]
  marks = @_ view(lines, 1:seperatorIndex - 1) |>
             map(split(_, ","), __)            |>
             map(parse.(Int, _), __)           |>
             map(map(v -> v + 1, _), __)
  folds = @_ view(lines, seperatorIndex + 1:length(lines)) |>
             map(split(_, "="), __)                        |>
             map([_[1], parse(Int, _[2]) + 1], __)
  (M=marks,F=folds)
end

#fn = "ExampleInput.txt"
fn = "SolutionInput.txt"
Data = parse_data(fn)

Paper = zeros(maximum(map(v -> v[2], Data.M)), maximum(map(v -> v[1], Data.M)))

for (x,y) ∈ Data.M
  Paper[CartesianIndex(y,x)] = 1
end

Paper

894×1311 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

## Part 1

In [133]:
function foldpaper_y(paper, index)
  Paper0 = view(copy(paper), 1:index - 1, :)
  Paper1 = reverse(view(copy(paper), index + 1:size(paper, 1), :); dims = 1)

  offset = 0
  if size(Paper1, 1) != size(Paper0, 1)
    offset = size(Paper0, 1) - size(Paper1, 1)
  end

  for index ∈ findall(v -> v > 0, Paper1)
    Paper0[index + CartesianIndex(offset, 0)] = 1
  end

  Paper0
end

function foldpaper_x(paper, index)
  Paper0 = view(copy(paper), :, 1:index - 1)
  Paper1 = reverse(view(copy(paper), :, index + 1:size(paper, 2)); dims = 2)

  offset = 0
  if size(Paper1, 2) != size(Paper0, 2)
    offset = size(Paper0, 2) - size(Paper1, 2)
  end

  for index ∈ findall(v -> v > 0, Paper1)
    Paper0[index + CartesianIndex(0, offset)] = 1
  end

  Paper0
end

@_ copy(Paper) |>
   reduce((paper, fold) -> begin
     @match fold[1] begin
      "fold along y" => foldpaper_y(paper, fold[2])
      "fold along x" => foldpaper_x(paper, fold[2])
    end
   end, [Data.F[1]]; init=__) |>
   count(v -> v > 0, __)


684

## Part 2

In [134]:
R = @_  copy(Paper) |>
        reduce((paper, fold) -> begin
          @match fold[1] begin
            "fold along y" => foldpaper_y(paper, fold[2])
            "fold along x" => foldpaper_x(paper, fold[2])
          end
        end, Data.F; init=__)

O = fill(".", size(R, 1), size(R, 2))

for index ∈ findall(v -> v > 0, R)
  O[index] = "#"
end

open("Solution.txt", "w") do io
  DelimitedFiles.writedlm(io, O)
end